In [1]:
import pandas as pd
import numpy as np

In [22]:
class NetPremium:
    
    def __init__(self, rate, i, SIZE=101, l0=10000):
        self.v = 1/(1+i)
        self.SIZE = SIZE
        self.l0 = l0
        self.q_x = rate['mortality'].groupby('구분').apply(lambda df: df[['연령', '위험률']].sort_values(by='연령')['위험률'].to_numpy())
        self.q_w = rate['lapse'].sort_values(by='시점')['해지율'].to_numpy()
        
    def get_product(self):
        """상품명 리턴"""
        
        return '참좋은간편건강보험1904'
        
    def get_coverage(self):
        """담보명 리턴"""
        
        return '질병1~5종수술비(동일질병당 1회지급)(간편고지)(갱신형, 비갱신형)'
    
    def __calc_maintainer(self, x, n, options):
        """유지자(순보험료 분자 부분) 계산"""
        
        # lx
        lx = np.zeros(self.SIZE)
        lx[0] = self.l0
        for i in range(self.SIZE-1):
            lx[i+1] = lx[i]*(1-self.q_w[i])

        # Cx, Mx
        C1x = lx*self.q_x['질병수술위로금위험률_질병1종수술비율']*self.v**(np.arange(self.SIZE)+0.5)*(1-self.q_w/2)
        C2x = lx*self.q_x['질병수술위로금위험률_질병2종수술비율']*self.v**(np.arange(self.SIZE)+0.5)*(1-self.q_w/2)
        C3x = lx*self.q_x['질병수술위로금위험률_질병3종수술비율']*self.v**(np.arange(self.SIZE)+0.5)*(1-self.q_w/2)
        C4x = lx*self.q_x['질병수술위로금위험률_질병4종수술비율']*self.v**(np.arange(self.SIZE)+0.5)*(1-self.q_w/2)
        C5x = lx*self.q_x['질병수술위로금위험률_질병5종수술비율']*self.v**(np.arange(self.SIZE)+0.5)*(1-self.q_w/2)
        
        M1x = C1x[::-1].cumsum()[::-1]
        M2x = C2x[::-1].cumsum()[::-1]
        M3x = C3x[::-1].cumsum()[::-1]
        M4x = C4x[::-1].cumsum()[::-1]
        M5x = C5x[::-1].cumsum()[::-1]
        
        if ((options['new'] and ~options['renewal']) or options['renewal']):
            return 0.5*((10/300)*C1x[x]+(30/300)*C2x[x]+(50/300)*C3x[x]+(100/300)*C4x[x]+C5x[x]) \
                + (10/300)*(M1x[x+1]-M1x[x+n])+(30/300)*(M2x[x+1]-M2x[x+n])+(50/300)*(M3x[x+1]-M3x[x+n])+(100/300)*(M4x[x+1]-M4x[x+n])+(M5x[x+1]-M5x[x+n])
        else:
            return (10/300)*(M1x[x]-M1x[x+n])+(30/300)*(M2x[x]-M2x[x+n])+(50/300)*(M3x[x]-M3x[x+n])+(100/300)*(M4x[x]-M4x[x+n])+(M5x[x]-M5x[x+n])
            
    
    def __calc_payer(self, x, m, mpp, options):
        """납입자(순보험료 분모 부분) 계산"""
        
        # 1종, 3종, 9종
        if options['types'] in ['1종', '3종', '9종']:
            q_j = list(map(lambda k: self.q_x[k], ['간편고지상해1급80%이상후유장해발생률', '간편고지질병80%이상후유장해발생률', '기타피부암및갑상선암이외의암발생률', '뇌졸중발생률', '급성심근경색증발생률']))
            
            # lpx
            lpx = np.zeros(self.SIZE)
            lpx[0] = self.l0
            for i in range(self.SIZE-1):
                lpx[i+1] = lpx[i]*(1-self.q_w[i])
                for j in range(len(q_j)):
                    Dj = 3/4 if j==2 and ((options['new'] and ~options['renewal']) or options['renewal']) and i == x else 1
                    lpx[i+1] *= 1-Dj*q_j[j][i]

            # Dpx, Npx
            Dpx = lpx*self.v**np.arange(self.SIZE)
            Npx = Dpx[::-1].cumsum()[::-1]

        # 5종, 7종, 11종
        elif options['types'] in ['5종', '7종', '11종']:

            # lpx
            lpx = np.zeros(self.SIZE)
            lpx[0] = self.l0
            for i in range(self.SIZE-1):
                lpx[i+1] = lpx[i]*(1-self.q_w[i])

            # Dpx, Npx
            Dpx = lpx*self.v**np.arange(self.SIZE)
            Npx = Dpx[::-1].cumsum()[::-1]

        else:
            raise Exception('종구분오류')
            
        return mpp*((Npx[x]-Npx[x+m])-(mpp-1)/(2*mpp)*(Dpx[x]-Dpx[x+m]))
    
    def calc_net_premium(self, x, n, m, mpp, options):
        """순보험료 계산"""
        
        return self.__calc_maintainer(x, n, options)/self.__calc_payer(x, m, mpp, options)

In [23]:
# 데이터 불러오기
mortality_rate = pd.read_excel('data/참좋은간편건강보험1904_기초데이터_20200805.xlsx', sheet_name='위험률', dtype={'연령': int, '위험률': float})
lapse_rate = pd.read_excel('data/참좋은간편건강보험1904_기초데이터_20200805.xlsx', sheet_name='해지율', dtype={'시점': int, '해지율': float})
rate = {'mortality': mortality_rate, 'lapse': lapse_rate}

In [24]:
# 테스트
net_premium = NetPremium(rate, 0.02)
x, n, m, mpp = 30, 50, 10, 12
100000000*net_premium.calc_net_premium(x, n, m, mpp, {'types': '5종', 'renewal': True, 'new': True})

23728.415447654657